# 식신 - 제주시청

In [77]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

### 1. 데이터 가져오기

In [78]:
site = 'https://www.siksinhot.com'
url = f'{site}/search?keywords={quote("제주시청")}'
req = requests.get(url)
html = req.text
#html

In [79]:
soup = BeautifulSoup(html, 'html.parser')

### 2. 찾으려고 하는 데이터의 태그 찾기

In [80]:
lis = soup.select_one('div.listTy1').find_all('li')
len(lis)

60

In [81]:
lis = soup.select('div.listTy1 > ul > li')      # 자식 Selector
len(lis)

12

### 3. 하나를 선택해서 원하는 정보 추출
    상호, 평점, 메뉴, 서브 페이지의 url
    전화번호, 주소

In [82]:
li = lis[1]
href = li.find('a')['href']
href

'/P/466268'

In [83]:
score = li.select_one('.score').string
score

'3.4'

In [84]:
store = li.select_one('.store').get_text()
store

'안양돈까스 제주시점'

In [85]:
menu = li.find('p').get_text()
menu

'돈까스 세트, 치즈돈까스 세트'

In [86]:
sub_url = site + href
sub_url

'https://www.siksinhot.com/P/466268'

In [87]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

In [88]:
tel = sub_soup.select_one('.p_tel').find('p').get_text()
tel

'064-799-1434'

In [89]:
addr = sub_soup.select_one('.txt_adr').get_text()
addr

'제주특별자치도 제주시 애월읍 애월로 40'

### 4. 페이지에 있는 데이터 가져오기

In [90]:
# try - except 를 이용해서 어디에서 에러가 발생했는지 확인
lines = []
for index, li in enumerate(lis):
    try:
        href = li.find('a')['href']
        score = li.select_one('.score').string
        store = li.select_one('.store').get_text()
        menu = li.find('p').get_text()

        sub_url = site + href
        sub_req = requests.get(sub_url)
        sub_html = sub_req.text
        sub_soup = BeautifulSoup(sub_html, 'html.parser')
        tel = sub_soup.select_one('.p_tel').find('p').get_text()
        addr = sub_soup.select_one('.txt_adr').get_text()
    except:
        print(index)

    lines.append([store, score, menu, tel, addr])

0


### 5. 에러난 부분을 처리할 수 있도록 수정

In [91]:
li = lis[0]
score = li.select_one('.cnt > em').get_text()
score

'평가중'

In [92]:
lis[1].select_one('.cnt > em').get_text()

'3.4'

### 6. 에러가 해결될 때 까지 5, 6을 반복

In [93]:
lines = []
for index, li in enumerate(lis):
    try:
        href = li.find('a')['href']
        score = li.select_one('.cnt > em').get_text()
        if score == '평가중':
            score = '0.0'
        store = li.select_one('.store').get_text()
        menu = li.find('p').get_text()

        sub_url = site + href
        sub_req = requests.get(sub_url)
        sub_html = sub_req.text
        sub_soup = BeautifulSoup(sub_html, 'html.parser')
        tel = sub_soup.select_one('.p_tel').find('p').get_text()
        addr = sub_soup.select_one('.txt_adr').get_text()
    except:
        print(index)

    lines.append([store, score, menu, tel, addr])

### 7. 데이터 프레임으로 만들기

In [94]:
df = pd.DataFrame(lines, columns=['업소명','평점','메뉴','전화번호','주소'])
df.head()

,업소명,평점,메뉴,전화번호,주소
0,제주시새우리,0.0,"딱새우김밥, 간장새우김밥, 양념새우컵밥, 딱새우꼬막무침, 한라봉에이드",064-900-2527,제주특별자치도 제주시 무근성7길 21
1,안양돈까스 제주시점,3.4,"돈까스 세트, 치즈돈까스 세트",064-799-1434,제주특별자치도 제주시 애월읍 애월로 40
2,올래국수,3.8,고기국수,064-742-7355,제주특별자치도 제주시 귀아랑길 24
3,명진전복,3.8,"전복죽, 전복돌솥밥, 전복구이, 전복회",064-782-9944,제주특별자치도 제주시 구좌읍 해맞이해안로 1282
4,삼성혈해물탕,3.9,"해물탕(특대), 해물탕(대), 해물탕(중), 해물탕(소), 전복회(소)",064-745-3000,제주특별자치도 제주시 선덕로5길 20


In [95]:
df.to_csv('식신맛집(제주시청).csv', index=False)